In [ ]:
!pip install huggingface huggingface-cli huggingface_hub torch torchvision torchaudio transformers

In [ ]:
import os

from tqdm import tqdm
from transformers import AutoTokenizer, pipeline, AutoModelForTokenClassification

In [ ]:
# !huggingface-cli login # do this in the terminal

In [ ]:
try:
    from google.colab import drive

    print(
        "You work on Colab. Gentle as we are, we will mount Drive for you. "
        "It'd help if you allowed this in the popup that opens."
    )
    drive.mount('/content/drive')
    DATA_DIR = os.path.join('drive', 'MyDrive', 'KEDiff', 'data')
except ModuleNotFoundError:
    print("You do not work on Colab")
    DATA_DIR = os.path.join('data')
print(f"{DATA_DIR=}", '-->', os.path.abspath(DATA_DIR))

In [ ]:
TOKENISER_CHECKPOINT = "dbmdz/bert-base-historic-multilingual-cased"
CHECKPOINT_NAME_BASE = "oalz-1788-q1-ner-"
TRAINED_DIR = os.path.join(DATA_DIR, 'trained_models', '2024-01-15')

In [ ]:
tokeniser = AutoTokenizer.from_pretrained(TOKENISER_CHECKPOINT)

In [ ]:
label_types = ["EVENT", "LOC", "MISC", "ORG", "PER", "TIME"]
selected_epochs = {
    "EVENT" : "checkpoint-1393",
    "LOC" : "checkpoint-1393",
    "MISC" : "checkpoint-2786",
    "ORG" : "checkpoint-1393",
    "PER" : "checkpoint-2786",
    "TIME" : "checkpoint-1393"
}
ner_model_paths = {
    label_type: os.path.join(TRAINED_DIR,
                             "".join([CHECKPOINT_NAME_BASE, label_type]),
                             selected_epochs[label_type])
    for label_type in label_types
}
ner_model_paths

In [ ]:
token_classifiers = {}
for label_type in tqdm(label_types):
    token_classifiers[label_type] = pipeline(
        "token-classification",
        model=os.path.abspath(ner_model_paths[label_type]),
        aggregation_strategy="simple"
    )
list(token_classifiers.keys())

In [ ]:
person_pipeline = token_classifiers["PER"]

In [ ]:
person_tokeniser = person_pipeline.tokenizer
person_model = person_pipeline.model

In [ ]:
# person_tokeniser.push_to_hub(repo_id="OALZ-1788-Q1-NER-PER")

In [ ]:
person_model.push_to_hub(repo_id="OALZ-1788-Q1-NER-PER")

In [ ]:
sample_text = "Dieses Modell soll nur eine der folgenden Kategorien erkennen: Personen-, Orts-, Organisations-, Ereignis-, Artefaktnamen oder Datumsangaben. Daher sprechen wir nun über Wilhelm Thell, der am 1. Jänner 1788, also noch vor der französischen Revolution und vor Vorstellung des ersten iPhones durch Apple im Jahre 2008, in Salzburg geboren wurde. Er studierte Pharmakologie an der Medizinischen Universität Wien."

In [ ]:
model_hf = AutoModelForTokenClassification.from_pretrained("LelViLamp/OALZ-1788-Q1-NER-PER")

In [ ]:
pipeline("ner", model=model_hf, tokenizer=person_tokeniser)(sample_text)